In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from IPython.display import display, Markdown
from collections import Counter
import random
from lightgbm import LGBMClassifier

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import lightning as L
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torchmetrics.classification import AUROC
from lightning.pytorch.callbacks import EarlyStopping

AttributeError: partially initialized module 'torch._dynamo' has no attribute 'disable' (most likely due to a circular import)

In [4]:
EXP_NAME = 'gemma_att'

In [5]:
os.getcwd()

'c:\\Users\\Piotr.Matys\\Python_Projects\\hallu_project\\Research\\notebooks'

In [6]:
context_df = None

for f_ in os.listdir(os.path.join("..", "..", EXP_NAME)):
    
    # if f_.startswith('attension'):

    if context_df is None:
        context_df = pd.read_parquet(os.path.join("..", "..", EXP_NAME, f_))
    else:
        context_df = pd.concat((context_df, pd.read_parquet(os.path.join("..", "..", EXP_NAME, f_))))

In [7]:
context_df['dataset'].value_counts()

dataset
cnndm          19400
nq              8030
xsum            7776
poquad_v2       6247
hotpotqa_en     3121
bioask          3056
hotpotqa_pl     2208
polqa           1869
Name: count, dtype: int64

In [8]:
context_df = context_df[context_df['dataset'].isin(['cnndm', 'nq'])]

In [9]:
context_df['label'].value_counts()

label
0    25940
1     1490
Name: count, dtype: int64

In [ ]:

# 1. Create a Synthetic Dataset
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data.iloc[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.long)


# 2. Define the LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=1):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.hidden_dim = hidden_dim

    def forward(self, x):
        x = x.unsqueeze(1)  # (batch_size, seq_len, input_dim)
        _, (hn, _) = self.lstm(x)  # hn is (num_layers, batch_size, hidden_dim)
        out = self.fc(hn[-1])  # Use the last layer's hidden state
        return out

# 3. Create the Lightning Module
class LSTMClassifier(L.LightningModule):
    def __init__(self, input_dim, hidden_dim, output_dim, lr=5e-4, num_layers=1):
        super().__init__()
        self.model = LSTMModel(input_dim, hidden_dim, output_dim, num_layers=num_layers)
        self.criterion = nn.CrossEntropyLoss()
        self.lr = lr

        # AUROC metric for binary classification
        self.train_auc = AUROC(task="binary")
        self.val_auc = AUROC(task="binary")

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        data, labels = batch
        preds = self(data)
        loss = self.criterion(preds, labels)
        # Calculate AUC during training
        prob = torch.softmax(preds, dim=1)[:, 1]  # Take probabilities for class 1
        auc = self.train_auc(prob, labels)
        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("train_auc", auc, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        data, labels = batch
        preds = self(data)
        loss = self.criterion(preds, labels)
        # Calculate AUC during validation
        prob = torch.softmax(preds, dim=1)[:, 1]  # Take probabilities for class 1
        auc = self.val_auc(prob, labels)
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_auc", auc, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)




In [ ]:
X_train = df_cnn.drop(columns=['label', 'dataset'])
y_train = df_cnn['label']

X_val = df_xsum.drop(columns=['label', 'dataset'])
y_val = df_xsum['label']

train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)

# 4. Train the Model
input_dim = X_train.shape[1]
hidden_dim = 64
output_dim = 2


model = LSTMClassifier(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, lr=1e-5, num_layers=2)

# 1. Set up EarlyStopping callback
early_stop_callback = EarlyStopping(
    monitor="val_loss",  # The metric to monitor
    patience=4,          # Number of epochs with no improvement after which training will stop
    verbose=True,        # Display a message when stopping
    mode="min",          # Minimize the monitored metric (for loss)
)

# 2. Initialize the Trainer with the early stopping callback
trainer = L.Trainer(
    max_epochs=100,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    log_every_n_steps=10,
    callbacks=[early_stop_callback],  # Add the early stopping callback here
)

# 3. Start the training
trainer.fit(model, train_loader, val_loader)



GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | model     | LSTMModel        | 60.8 K | train
1 | criterion | CrossEntropyLoss | 0      | train
2 | train_auc | BinaryAUROC      | 0      | train
3 | val_auc   | BinaryAUROC      | 0      | train
-------------------------------------------------------
60.8 K    Trainable params
0         Non-trainable params
60.8 K    Total params
0.243     Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode
SLURM auto-requeueing enabled. Setting signal handlers.


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/net/tscratch/people/plgkonkie311/miniconda3/envs/hallu/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


/net/tscratch/people/plgkonkie311/miniconda3/envs/hallu/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Epoch 0:   0%|          | 0/244 [00:00<?, ?it/s] 

...

Epoch 7: 100%|██████████| 244/244 [00:17<00:00, 14.19it/s, v_num=23, val_loss=0.337, val_auc=0.523, train_loss=0.172, train_auc=0.325]

Metric val_loss improved by 0.001 >= min_delta = 0.0. New best score: 0.337


Epoch 11: 100%|██████████| 244/244 [00:26<00:00,  9.32it/s, v_num=23, val_loss=0.358, val_auc=0.525, train_loss=0.133, train_auc=0.318]

Monitored metric val_loss did not improve in the last 4 records. Best score: 0.337. Signaling Trainer to stop.


Epoch 11: 100%|██████████| 244/244 [00:26<00:00,  9.31it/s, v_num=23, val_loss=0.358, val_auc=0.525, train_loss=0.133, train_auc=0.318]
